In [2]:
import pandas as pd
import os
import ast
import re
import random
import json

In [3]:
json_brands = 'Data/brands.json'
brands = []
with open(json_brands, 'r') as j:
     brands_json = json.loads(j.read())['RECORDS']
for i in range(len(brands_json)):
    brands.append(brands_json[i]['name'])
json_devices = 'Data/devices.json'
devices = []
with open(json_devices, 'r') as j:
     devices_json = json.loads(j.read())['RECORDS']
for i in range(len(devices_json)):
    devices.append(devices_json[i]['name'])

In [19]:
only_files = [f for f in os.listdir("VLSP23-Comparative-Opinion-Mining/data/relabledData") if
              os.path.isfile(os.path.join("VLSP23-Comparative-Opinion-Mining/data/relabledData", f))]
type_of_compare = ["DIF", "EQL", "SUP+", "SUP-", "SUP", "COM+", "COM-", "COM"]
data_frame = {"content": [], "comparative": [], "subject": [], "object": [], "aspect": [], "predicate": [], "label": [], "NER": []}
pred_label = dict()

In [20]:
def preprocessing_with_BIO(filename):
    data_row = []
    labeled_row = dict()
    data = []
    with (open('Data/VLSP2023_ComOM_training_v2/VLSP2023_ComOM_training_v2/' + filename, 'r', encoding="utf-8") as f):
        row = f.readline()
        while row != "":
            split_row = row.split("\t")
            if len(split_row) > 1:
                r = split_row[1]
            else:
                r = row
            r = r.replace("\n", "")
            data_row.append(r)
            row = f.readline()
    f.close()
    filename = filename[:-4]
    del f
    j = 0
    for i in range(len(data_row) - 1, -1, -1):
        if data_row[i] == "":
            data_row.remove(data_row[i])
    for i in range(len(data_row) - 1):
        if re.match("^\{\"subject\": \[", data_row[i + 1]) and re.match("^\{\"subject\": \[", data_row[i]):
            row = eval(data_row[i + 1])
            labeled_row[data_row[j]].append(row)
        elif re.match("^\{\"subject\": \[", data_row[i + 1]):
            j = i
            row = eval(data_row[i + 1])
            labeled_row[data_row[j]] = [row]
    for i in data_row:
        if re.match("^\{\"subject\": \[", i) is None:
            data.append(i)
    del data_row
    del j
    del row
    for i in data:
        sentence_label = []
        key = 0
        subj = []
        obj = []
        asp = []
        pred = []
        lab = []
        w = i.split(" ")
        label = [0 for k in range(len(w))]
        if i in labeled_row.keys():
            key = 1
            for k in range(len(labeled_row.get(i))):
                list_dct = labeled_row.get(i)
                if type(list_dct[k]["subject"]) == str:
                    continue
                for j in list_dct[k].keys():
                    arr = list_dct[k].get(j)
                    consecutive = []
                    if j == "label":
                        continue
                    for e in range(len(arr)):
                        consecutive.append(int(arr[e][:arr[e].find("&")]))
                        arr[e] = re.sub("^[0-9]+&&", "", arr[e])
                    s = " ".join(arr)
                    list_dct[k][j] = s
                    for q in range(len(consecutive)):
                        if j == "subject":
                            if q == 0:
                                label[consecutive[q] - 1] = 1
                            else:
                                label[consecutive[q] - 1] = 2
                        elif j == "object":
                            if q == 0:
                                label[consecutive[q] - 1] = 3
                            else:
                                label[consecutive[q] - 1] = 4
                        elif j == "aspect":
                            if q == 0:
                                label[consecutive[q] - 1] = 5
                            else:
                                label[consecutive[q] - 1] = 6
                        elif j == "predicate":
                            if q == 0:
                                label[consecutive[q] - 1] = 7
                            else:
                                label[consecutive[q] - 1] = 8
                sentence_label.append(label)
                subj.append(list_dct[k]["subject"])
                obj.append(list_dct[k]["object"])
                pred.append(list_dct[k]["predicate"])
                asp.append(list_dct[k]["aspect"])
                lab.append(type_of_compare.index(list_dct[k]["label"]) + 1)
            if len(sentence_label) > 1:
                full_label = []
                l = 0
                for n in range(len(sentence_label[0])):
                    for j in range(len(sentence_label) - 1):
                        if sentence_label[j][n] == sentence_label[j + 1][n]:
                            l = sentence_label[j][n]
                        elif sentence_label[j][n] != sentence_label[j + 1][n]:
                            if sentence_label[j][n] == 0:
                                l = sentence_label[j + 1][n]
                            elif sentence_label[j + 1][n] == 0:
                                l = sentence_label[j][n]
                    full_label.append(l)
                sentence_label = full_label
                del full_label
            else:
                sentence_label = label
        else:
            sentence_label = label
        data_frame["content"].append(i)
        data_frame["comparative"].append(key)
        data_frame["subject"].append(subj)
        data_frame["object"].append(obj)
        data_frame["predicate"].append(pred)
        data_frame["aspect"].append(asp)
        data_frame["label"].append(lab)
        data_frame["NER"].append(sentence_label)
        for j in range(len(pred)):
            if pred[j] not in pred_label.keys():
                pred_label[pred[j]] = [lab[j]]
            elif pred[j] in pred_label.keys() and lab[j] not in pred_label[pred[j]]:
                pred_label[pred[j]].append(lab[j])
    return data_frame
for i in only_files:
    preprocessing_with_BIO(i)

In [21]:
df_data = pd.DataFrame(data_frame)
df_data.head()

,content,comparative,subject,object,aspect,predicate,label,NER
0,"Title : Review iPhone 14 : Thiết kế , hiệu năn...",0,[],[],[],[],[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,"iPhone 14 được ra mắt vào 09 / 2022 , được đán...",1,"[iPhone 14, iPhone 14]","[thế hệ trước, thế hệ trước]","[phần cứng, phần mềm]","[nâng cấp nhẹ, nâng cấp nhẹ]","[6, 6]","[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Về ngoại hình thì cũng có rất ít sự khác biệt ...,1,[],[thế hệ tiền nhiệm],[ngoại hình],[rất ít sự khác biệt],[2],"[0, 5, 6, 0, 0, 0, 7, 8, 8, 8, 8, 0, 0, 3, 4, ..."
3,"Có một vài ý kiến trái chiều cho rằng , phiên ...",0,[],[],[],[],[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Thực tế khi mua hoặc nâng cấp một thiết bị di ...,0,[],[],[],[],[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [24]:
random.seed(1)
counter = 0
stat = [0,0,0,0,0,0,0,0,0,0]
vocab = {'subject':set(),'object':set(),'aspect':set(),'predicate':dict()}
dct_split = {'content':[],'subject':[],'label':[],'object':[],'aspect':[],'predicate':[], 'NER':[]}
lst = compared.to_dict(orient= "list")
pred_stat = dict()
overlap = dict()
for i in range(len(lst['content'])):
    subject = ast.literal_eval(lst['subject'][i])
    labels = ast.literal_eval(lst['label'][i])
    object = ast.literal_eval(lst['object'][i])
    aspect = ast.literal_eval(lst['aspect'][i])
    predicate = ast.literal_eval(lst['predicate'][i])
    ner = ast.literal_eval(lst['NER'][i])
    for j in range(len(labels)):
        n_subject = ' '.join(subject[j].split('_'))
        n_object = ' '.join(object[j].split('_'))
        n_aspect = ' '.join(aspect[j].split('_'))
        pred = ' '.join(predicate[j].split('_')).lower()
        dct_split['content'].append(lst['content'][i])
        dct_split['subject'].append(n_subject)
        dct_split['object'].append(n_object)
        dct_split['aspect'].append(n_aspect)
        dct_split['predicate'].append(pred)
        dct_split['label'].append(labels[j])
        dct_split['NER'].append(ner)
        vocab['subject'].add(n_subject)
        vocab['object'].add(n_object)
        vocab['aspect'].add(n_aspect)
        stat[labels[j]] += 1
        if pred in vocab['predicate'].keys() and labels[j] not in vocab['predicate'][pred]:
            vocab['predicate'][pred].append(labels[j])
        else:
            vocab['predicate'][pred]=[labels[j]]
        # Predicate statistics on labels
        if pred in pred_stat.keys():
            pred_stat[pred][labels[j]] += 1
        else:
            pred_stat[pred]=[0,0,0,0,0,0,0,0,0]
            pred_stat[pred][labels[j]]+=1
        # Overallping labels
        if n_object in pred and n_object != '':
            if pred in overlap.keys():
                overlap[pred].append('ob_'+n_object)
            else:
                overlap[pred] = ['ob_'+n_object]
        if n_subject in pred and n_subject != '':
            if pred in overlap.keys():
                overlap[pred].append('sub_'+n_subject)
            else:
                overlap[pred] = ['sub_'+n_subject]
        if n_aspect in pred and n_aspect != '':
            if pred in overlap.keys():
                overlap[pred].append('asp_'+n_aspect)
            else:
                overlap[pred] = ['asp_'+n_aspect]
df_splited = pd.DataFrame(data=dct_split)
for i in brands:
    vocab['subject'].add(i)
    vocab['object'].add(i)
for i in devices:
    vocab['subject'].add(i)
    vocab['object'].add(i)

In [10]:
def finder(stg, finder):
    split = stg.split()
    start = -1
    end = -1
    for s in range(len(split)):
        if split[s] in finder and start == -1 and end == -1:
            start = s
            end = s
        elif split[s] in finder and start != -1 and end != -1:
            end = s
        elif split[s] not in finder and start != -1 and end != -1:
            return start, end
    return start, end

In [11]:
data = {'content':[], 'NER':[], 'label':[]}
generror = []
for k in range(1,9):
    df_comp = df_splited[df_splited['label'] == k]
    others = df_comp[['subject','object','aspect','NER']]
    content = df_comp.to_dict(orient='list')['content']
    pred = df_comp.to_dict(orient='list')['predicate']
    i = 0
    count = 0
    while count < 1000:
        stg = content[i] 
        su = others.iloc[i]['subject']
        ob = others.iloc[i]['object']
        asp = others.iloc[i]['aspect']
        pr = pred[i]
        n = others.iloc[i]['NER']
        if su != '' and su in stg:
            f = su.split()
            sub = random.choice(list(vocab['subject']))  
            substitute = sub.split()
            checkpoint = finder(stg,f)
            stg = stg.replace(su, sub)
            rep = [1]+[2 for p in range(len(substitute)-1)]
            n = n[:checkpoint[0]]+rep+ n[checkpoint[1]+1:]
            if len(stg.split()) != len(n):
                generror.append(stg+','+n.__str__()+','+str(k)+','+su+','+sub+',sub,'+str(checkpoint))
        if ob != '' and ob in stg:
            f = ob.split()
            obj = random.choice(list(vocab['object']))
            substitute = obj.split()
            checkpoint = finder(stg, f)
            stg = stg.replace(ob, obj)
            rep = [3]+[4 for p in range(len(substitute)-1)]
            n = n[:checkpoint[0]]+rep+ n[checkpoint[1]+1:]
            if len(stg.split()) != len(n):
                generror.append(stg+','+n.__str__()+','+str(k)+','+ob+','+obj+',ob,'+str(checkpoint))
        if asp != '' and asp in stg:
            f = asp.split()
            aspe = random.choice(list(vocab['aspect']))
            substitute = aspe.split()
            checkpoint = finder(stg,f)
            stg = stg.replace(asp, aspe)
            rep = [5]+[6 for p in range(len(substitute)-1)]
            n = n[:checkpoint[0]]+rep+ n[checkpoint[1]+1:]
            if len(stg.split()) != len(n):
                generror.append(stg+','+n.__str__()+','+str(k)+','+asp+','+aspe+',asp,'+str(checkpoint))
        if stg not in content:
            data['content'].append(stg)
            data['NER'].append(n)
            data['label'].append(k)
            count += 1
        i += 1
        if i == stat[k]:
            i = 0

Khúc sau là statistic một xíu của data thì m muốn xem qua thì xem

In [25]:
print(overlap)

{'hiếm có chiếc điện thoại nào trên thị trường có thể làm được': ['ob_chiếc điện thoại nào trên thị trường'], 'nhiều tính năng vượt trội hơn': ['asp_tính năng'], 'đều là những chiếc smartphone trang bị camera tốt nhất hiện nay': ['asp_camera'], 'tương đối tốt ở cả hai máy': ['ob_cả hai máy', 'sub_cả hai máy'], 'nâng cấp chất lượng pin hơn': ['asp_chất lượng pin'], 'nhiều chức năng thực dụng hơn': ['asp_chức năng thực dụng'], 'nhiều lợi ích hơn': ['asp_lợi ích'], 'sự chênh lệch về các thông số khác như camera cũng không đáng kể': ['asp_các thông số khác như camera'], 'dễ thao tác hơn': ['asp_thao tác'], 'sự khác biệt về hiệu suất thực tế là không đáng kể': ['asp_hiệu suất thực tế'], 'tốt nhất trong bộ ba điện thoại ngày hôm nay': ['ob_bộ ba điện thoại ngày hôm nay'], 'chiếm nhiều không gian lưu trữ hơn': ['asp_không gian lưu trữ'], 'nâng cao chi tiết ở những vùng tối hơn': ['asp_chi tiết ở những vùng tối'], 'không smartphone kiểu dáng thanh nào ngày nay có thể làm được': ['sub_smartphon

In [41]:
print(stat)

[0, 58, 286, 106, 5, 4, 499, 107, 20, 0]


In [40]:
counter = 0
counter1 = 0
counter2 = 0
counter3 = 0
counter4 = 0
counter5 = 0
for i in pred_stat:
    if sum(pred_stat[i]) < 2:
        counter += 1
    if sum(pred_stat[i]) > 1 and sum(pred_stat[i]) < 5:
        counter1 += 1
    if sum(pred_stat[i]) > 4 and sum(pred_stat[i]) < 10:
        counter2 += 1
    if sum(pred_stat[i]) > 9 and sum(pred_stat[i]) < 20:
        counter3 += 1
    if sum(pred_stat[i]) > 19 and sum(pred_stat[i]) < 30:
        counter4 += 1
    if sum(pred_stat[i]) > 30:
        counter5 += 1
print(counter)  
print(counter1)
print(counter2)  
print(counter3)
print(counter4)
print(counter5)

472
127
18
5
3
1


In [43]:
for i in pred_stat:
    if pred_stat[i][8] > 0:
        print(i,pred_stat[i])

giảm bớt [0, 0, 0, 0, 0, 0, 0, 0, 1]
nhỏ gọn hơn [0, 0, 0, 0, 0, 0, 2, 0, 1]
nặng hơn [0, 0, 0, 0, 0, 0, 1, 1, 1]
rộng hơn [0, 0, 0, 0, 0, 0, 6, 0, 1]
sáng hơn [0, 0, 0, 0, 0, 0, 3, 0, 2]
nhỏ gọn hơn [0, 0, 0, 0, 0, 0, 0, 0, 1]
lớn và nặng hơn [0, 0, 0, 0, 0, 0, 0, 0, 1]
tối hơn [0, 0, 0, 0, 0, 0, 0, 0, 1]
độc đáo hơn [0, 0, 0, 0, 0, 0, 0, 0, 1]
hơn hẳn [0, 0, 0, 0, 0, 0, 4, 0, 1]
to hơn [0, 0, 0, 0, 0, 0, 1, 0, 2]
vuông vắn hơn [0, 0, 0, 0, 0, 0, 0, 0, 1]
nông hơn trong một số bức ảnh [0, 0, 0, 0, 0, 0, 0, 0, 1]
nhô lên hơi cao so với [0, 0, 0, 0, 0, 0, 0, 0, 1]
nhỉnh hơn đôi chút [0, 0, 0, 0, 0, 0, 0, 0, 1]
“ truyền thống ” hơn [0, 0, 0, 0, 0, 0, 0, 0, 1]
ít cong hơn [0, 0, 0, 0, 0, 0, 0, 0, 1]
điều chỉnh hơn nữa [0, 0, 0, 0, 0, 0, 0, 0, 1]


In [29]:
prev_pred = dict() 
for i in range(len(df_splited)):
    if df_splited['predicate'].iloc[i] not in prev_pred.keys():
        prev_pred[df_splited['predicate'].iloc[i]] = {'sentence':df_splited['content'].iloc[i],'label':[df_splited['label'].iloc[i]]}
    elif df_splited['predicate'].iloc[i] in prev_pred.keys():
        if prev_pred[df_splited['predicate'].iloc[i]]['sentence'] == df_splited['content'].iloc[i] :
            prev_pred[df_splited['predicate'].iloc[i]]['label'].append(df_splited['label'].iloc[i])
for i in prev_pred:
    if len(prev_pred[i]['label'])  > 1:
        print(prev_pred[i])

{'sentence': 'iPhone 14 được ra mắt vào 09 / 2022 , được đánh giá là chỉ nâng cấp nhẹ về phần cứng cũng như phần mềm so với thế hệ trước .', 'label': [6, 6]}
{'sentence': 'Nếu so sánh với một số chipset khác như Dimensity 9000 hay Snapdragon 8 Gen 1 thì có hơi thấp một chút .', 'label': [7, 7]}
{'sentence': 'Số điểm này cao hơn rất nhiều so với Snapdragon 8 Gen 1 ( 3.578 điểm đa nhân và 1.238 điểm đơn nhân ) và Dimensity 9000 ( 4.207 điểm đa nhân và 1.264 điểm đơn nhân ) .', 'label': [6, 6]}
{'sentence': 'Tuy nhiên camera siêu rộng và camera tele hầu như không nhận được nâng cấp nào về phần cứng , chính vì vậy trong điều kiện thiếu sáng bộ đôi camera này lộ rõ nhiều nhược điểm .', 'label': [2, 2]}
{'sentence': 'Nhìn chung cả 2 chiếc điện thoại này không có sự khác biệt lớn về thiết kế và màu sắc .', 'label': [2, 2]}
{'sentence': 'Còn với Samsung Galaxy A53 mặc dù có các hiệu năng và khả năng hiển thị thấp hơn nhưng vẫn đáp ứng đủ nhu cầu cho các bạn giống như Samsung Galaxy A73 .', 'la

In [33]:
new_pred = []
for i in vocab['predicate']:
    if len(i.split()) < 3 and 'hơn' in i:
        new_pred.append(i.replace("hơn",'nhất'))
print(new_pred)

['tốt nhất', 'cao nhất', 'khá nhất', 'lớn nhất', 'rẻ nhất', 'nhiều nhất', 'thấp nhất', 'nặng nhất', 'rộng nhất', 'nhỏ nhất', 'lâu nhất', 'lớn nhất', 'tốt nhất', 'thấp nhất', 'mát nhất', 'ấm nhất', 'sáng nhất', 'tối nhất', 'kém nhất', 'nhất', 'nhẹ nhất', 'mới nhất', 'sáng nhất', 'nhanh nhất', 'nhất hẳn', 'to nhất', 'nhỉnh nhất', 'mỏng nhất', 'chậm nhất', 'yếu nhất', 'đẹp nhất', 'rõ nhất', 'dài nhất', 'mạnh nhất', 'ít nhất']


In [12]:
for i in range(len(data['content'])-1,-1,-1):
    if len(data['content'][i].split()) != len(data['NER'][i]):
        data['content'].pop(i)
        data['NER'].pop(i)
        data['label'].pop(i)

In [13]:
counter = [0,0,0,0,0,0,0,0,0]
count = 0
for i in range(len(data['content'])):
    counter[data['label'][i]] += 1
print(counter)
print(stat)

[0, 788, 841, 828, 607, 808, 809, 824, 893]
[0, 58, 286, 106, 5, 4, 499, 107, 20, 0]


In [14]:

df_prev = pd.read_csv('Data/total.csv',encoding='utf-8')
dct_prev = df_prev[df_prev['comparative'] == 1].to_dict(orient='list')
counter = 0
dct_total = {'content':dct_split['content'],'NER':dct_split['NER'],'label':dct_split['label']}
for i in range(len(data['content'])):
    if stat[data['label'][i]] > 600:
        continue
    dct_total['content'].append(data['content'][i])
    dct_total['NER'].append(data['NER'][i])
    dct_total['label'].append(data['label'][i])
    stat[data['label'][i]]+=1


In [15]:
df_generate = pd.DataFrame(data=dct_total)
df_shuffled = df_generate.sample(len(df_generate), random_state=1)
df_shuffled.to_csv('Data/combined_BIO.csv')

In [37]:
import csv  
# name of csv file  
rows = []
filename = "combined.csv"
with open(filename, 'w', encoding="utf-8") as csvfile:  
    csvwriter = csv.writer(csvfile)  
    csvwriter.writerow(["Content","NER","Label"])  
    for i in range(len(dct_total['content'])):
        row = [dct_total['content'][i]]
        row.append(dct_total['NER'][i])
        row.append(dct_total['label'][i])
        rows.append(row)
    csvwriter.writerows(rows) 

In [39]:
d = pd.read_csv('combined.csv',encoding='utf-8')
d.head()

,Content,NER,Label
0,"iPhone 14 được ra mắt vào 09 / 2022 , được đán...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[6, 6]"
1,Về ngoại hình thì cũng có rất ít sự khác biệt ...,"[0, 3, 3, 0, 0, 0, 4, 4, 4, 4, 4, 0, 0, 2, 2, ...",[2]
2,"Nhìn chung , về thiết kế của iPhone 14 thì khô...","[0, 0, 0, 0, 3, 3, 0, 1, 1, 0, 4, 4, 4, 4, 4, ...",[2]
3,"Tương tự , thì hệ thống camera vẫn giữ nguyên ...","[0, 0, 0, 0, 1, 1, 1, 0, 4, 4, 3, 3, 0, 0, 0, ...",[2]
4,"So với iPhone 13 , thì rất khó để nhận ra sự k...","[0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 0, ...","[2, 2]"
